In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
from qutip import *
from helpers.estimation import *
from helpers.data_processing import *
from helpers.targets import *
import exp_params

USING ORENS OBSERVABLES


### Check yourself: print function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

Q
D=6
nD=35


### State list

In [4]:
state_list = exp_params.cat_state_list
print(state_list)

['cat-eve-1' 'cat-nmp-1' 'cat-nop-1' 'cat-odd-1']


### Experimental data directory

In [7]:
exp_data_directory = r"data\fidelity_v_t2\Q"
all_files = np.array(os.listdir(exp_data_directory))

### Directory for target states generated from grape (we don't use ideal when we're processing experimental data)

In [8]:
# GRAPE generated target states
target_states_directory = r"target_states"

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [9]:
# Load inverse map variables for converting pe to rho
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_Q.npz"
)
W = map_variables["W"]
beta = map_variables["beta"]


### Top level iteration through the T2s. Then, iterate through the states. For each state: 1) collect the observable outcomes, 2) perform linear inversion with the map to get $\rho_{LS}$, 3) perform both MLE and Bayesian, and 4) record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each.

In [10]:
F_MLE = np.zeros(len(state_list), dtype=float)
F_Bayes_BME = np.zeros(len(state_list), dtype=float)

final_rho_BME = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)

final_rho_MLE = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)


In [12]:
RR_No_list = [0, 1, 2, 3, 4]


for rrno in RR_No_list:
    print(f"T2: {rrno}\n\n\n")
    for j, state_name in enumerate(state_list):  # State List

        data = np.zeros([exp_params.nD])  # Initialize data vector


        for point in range(1, exp_params.nD + 1):  # Point1 to Point35

            ending = (
                "_grape_" + str(state_name) + "_point" + str(point - 1) + f"_No{rrno}"
            )
            matching = [
                filename
                for filename in all_files
                if (filename.split("_ampx")[0]).endswith(str(ending))
            ]
            filepath = exp_data_directory + "/" + matching[0]
            file = h5py.File(filepath, "r")


            signal = post_selection(filepath)


            if state_name in exp_params.pes_after_grape[:, 0]:

                pge = exp_params.pes_after_grape[

                    (exp_params.pes_after_grape[:, 0] == state_name), 1

                ].astype(float)


            else:

                pge = exp_params.pe_after_grape_avg


            aux = (signal - pge) / (1 - 2 * pge)

            data[point - 1] = aux


        rho_est, qRho_est = get_LS_and_MLE_rho_est(

            data, W, beta, exp_params.D, exp_params.nD
        )


        rho_tar_qc = Y_target(

            state_name, target_states_directory, qdim=exp_params.qdim, cdim=exp_params.cdim
        )


        rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])

        rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()


        Fmean_bayes, Fstd_bayes, rho_BME = bayesian_rho_est(

            numSamp=2**10,

            N=exp_params.N_exp,

            rho_tar=rho_tar_c,

            rhoLS=rho_est.full(),
        )


        F_Bayes_BME[j] = fidelity(rho_tar_c, Qobj(rho_BME)) ** 2

        F_MLE[j] = fidelity(rho_tar_c, qRho_est) ** 2


        final_rho_BME[j, :, :] = rho_BME

        final_rho_MLE[j, :, :] = qRho_est


        print(f"state {j}: {state_name}: {F_Bayes_BME[j]}")
    np.savez(
        f"results_t2\exp\Q\Bayes_F_rrno={rrno}_BME_MLE_rho_Q.npz",
        F_Bayes_BME=F_Bayes_BME,
        F_MLE=F_MLE,
        final_rho_BME=final_rho_BME,
        final_rho_MLE=final_rho_MLE,
    )

T2: 0





C:\Users\clara\AppData\Local\Temp\ipykernel_21864\673795003.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data[point - 1] = aux


state 0: cat-eve-1: 0.9481592423310247


### Print and save results in an npz